In [20]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
# load the training data
train_data = pd.read_csv('train.csv')

In [22]:
# define hyperparameters
batch_size = 128
noise_dim = 100
epochs = 50

In [23]:
# create a discriminator model
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(train_data.shape[1],)))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [24]:
# create a generator model
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(128, activation='relu', input_dim=noise_dim))
    model.add(tf.keras.layers.Dense(train_data.shape[1], activation='sigmoid'))
    return model

In [25]:
# combine both models
def make_gan_model(generator, discriminator):
    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [26]:
discriminator = make_discriminator_model()
generator = make_generator_model()
gan_model = make_gan_model(generator, discriminator)

In [27]:
# train the model
for epoch in range(epochs):
    # generate noise
    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    # generate fake data
    fake_data = generator.predict(noise)
    # concatenate real and fake data
    real_data = train_data.iloc[np.random.randint(0, train_data.shape[0], batch_size)]
    data = np.concatenate((fake_data, real_data), axis=0)
    labels = np.concatenate((np.zeros((batch_size, 1)), np.ones((batch_size, 1))), axis=0)
    # train the discriminator
    d_loss, d_acc = discriminator.train_on_batch(data, labels)
    # generate noise
    noise = np.random.normal(0, 1, (batch_size, noise_dim))
    # generate labels
    labels = np.ones((batch_size, 1))
    # train the gan
    g_loss, g_acc = gan_model.train_on_batch(noise, labels)
    
    print('Epoch %d: [Discriminator loss: %f, Acc: %f], [Generator loss: %f, Acc: %f]' % (epoch, d_loss, d_acc, g_loss, g_acc))
    

4/4 [==============================] - 0s 2ms/step
Epoch 0: [Discriminator loss: 44.819523, Acc: 0.515625], [Generator loss: 0.865253, Acc: 0.007812]
4/4 [==============================] - 0s 3ms/step
Epoch 1: [Discriminator loss: 4.368700, Acc: 0.175781], [Generator loss: 0.505936, Acc: 1.000000]
4/4 [==============================] - 0s 2ms/step
Epoch 2: [Discriminator loss: 0.625100, Acc: 0.500000], [Generator loss: 0.324564, Acc: 1.000000]
4/4 [==============================] - 0s 2ms/step
Epoch 3: [Discriminator loss: 0.837041, Acc: 0.500000], [Generator loss: 0.210845, Acc: 1.000000]
4/4 [==============================] - 0s 1ms/step
Epoch 4: [Discriminator loss: 1.031958, Acc: 0.500000], [Generator loss: 0.141543, Acc: 1.000000]
4/4 [==============================] - 0s 675us/step
Epoch 5: [Discriminator loss: 1.213422, Acc: 0.500000], [Generator loss: 0.100779, Acc: 1.000000]
4/4 [==============================] - 0s 3ms/step
Epoch 6: [Discriminator loss: 1.382019, Acc: 0.50000

In [28]:
# generate artificial data
noise = np.random.normal(0, 1, (1000, noise_dim))
fake_data = generator.predict(noise)

32/32 [==============================] - 0s 1ms/step


In [29]:
# save data
pd.DataFrame(fake_data).to_csv('test.csv', index=False)